<a href="https://colab.research.google.com/github/mctrinh/ggColab/blob/main/starting_colab_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/"   # reach the drive at anytime
!cd                               # bash commands start with !
                                  # !python script.py to run a python file
                                  # !git clone <REPOSITORY URL> to clone a repository

In [ ]:
# download and unzip a shared zip file link
# !wget <ENTER URL>
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

In [14]:
# Import resources
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import json
import copy

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL
# print(PIL.PILLOW_VERSION)

from PIL import Image
from collections import OrderedDict

import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
# !pip3 install torch torchvision
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

import os

# install library: !pip install <LIBRARY> or !apt-get install <LIBRARY>
# !pip install -q keras
import keras

In [ ]:
# upload files from computer
from google.colab import files
files.upload()

In [ ]:
# download files and run
from google.colab import files
files.download('/content/filename.pdf')

In [ ]:
# default hardware in Colab is CPU
# to enable GPU/TPU: Runtime → Change runtime type → Hardware accelerator
# to check if GPU is enable:
import tensorflow as tf
tf.test.gpu_device_name()